In [1]:
load('../../../data/impactAnnotated.RData')

In [2]:
impact.ann.df$class_label <- impact.ann.df$confidence_class %in% c('MANUAL_OK', 'AUTO_OK')

In [3]:
# subset only the clearly labelled variants
impact.ann.subset.df <- subset(impact.ann.df, confidence_class %in% c('UNLIKELY', 'MANUAL_OK', 'AUTO_OK'))

In [4]:
labelTrain.subset <- impact.ann.subset.df$class_label 

In [5]:
table(impact.ann.subset.df$confidence_class)


  AUTO_OK MANUAL_OK  UNLIKELY 
   191228      7113     15317 

In [48]:
#colnames(impact.ann.subset.df)

In [6]:
selFeatures <- c('Consequence',
                 'cgType',
                #'t_depth',
                 't_vaf',
                 't_alt_count',
                 'n_depth',
                 'n_vaf',
                 'n_alt_count',
                 't_ref_plus_count',
                 't_ref_neg_count',
                 'occurence_in_normals',
                 #'minor_contamination',
                 'SIFT_score',
                 #'Polyphen2_HDIV_score',
                 #'LRT_score',
                 #'MutationTaster_score',
                 #'MutationAssessor_score',
                 #'FATHMM_score',
                 #'PROVEAN_score',
                 #'VEST3_score',
                 #'CADD_raw',
                 #'DANN_score',
                 #'fathmm.MKL_coding_score',
                 #'MetaSVM_score',
                 #'MetaLR_score',
                 #'integrated_fitCons_score',
                 #'X1000g2015aug_all',
                 'Kaviar_AF',
                 'isOncogene',
                 'Total.onco',
                 'cosmicCount'
                )
variantTrain <- impact.ann.subset.df [, selFeatures]
nrow(variantTrain)

[1] 213658

In [7]:
variantTrain.corr <- variantTrain
variantTrain.corr$Consequence <- as.factor(variantTrain.corr$Consequence)
variantTrain.corr$t_vaf <- as.numeric(variantTrain.corr$t_vaf)
variantTrain.corr$t_alt_count <- as.numeric(variantTrain.corr$t_alt_count)
variantTrain.corr$n_depth <- as.numeric(variantTrain.corr$n_depth)
variantTrain.corr$t_ref_plus_ratio<- as.numeric(variantTrain.corr$t_ref_plus_count)  / 
(as.numeric(variantTrain.corr$t_ref_plus_count) + as.numeric(variantTrain.corr$t_ref_neg_count))

variantTrain.corr$SIFT_score<- as.numeric(variantTrain.corr$SIFT_score)
variantTrain.corr$Kaviar_AF<- as.numeric(variantTrain.corr$Kaviar_AF)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [8]:
variantTrain.corr$occurence_in_normals <- as.numeric(as.character(gsub(';.*', '', variantTrain.corr$occurence_in_normals)))

In [9]:
t <- as.numeric(as.character(gsub(';.*', '', variantTrain.corr$occurence_in_normals)))
t[is.na(t)] <- 0
variantTrain.corr$occurence_in_normals <- t

In [10]:
variantTrain.corr$SIFT_score[is.na(variantTrain.corr$SIFT_score)] <- 1

In [11]:
variantTrain.corr$t_ref_plus_ratio[is.na(variantTrain.corr$t_ref_plus_ratio)] <- 0.5

In [12]:
variantTrain.corr$Kaviar_AF[is.na(variantTrain.corr$Kaviar_AF)] <- 0

In [13]:
table(variantTrain.corr$Total.onco)


    Inconclusive   Likely Neutral Likely Oncogenic        Oncogenic 
             466              248            42378            14558 
            null      NotInOncoKb 
               0           156008 

In [14]:
table(variantTrain.corr$isOncogene)
levels(variantTrain.corr$isOncogene) <- c(levels(variantTrain.corr$isOncogene), 'unclear')
variantTrain.corr$isOncogene[is.na(variantTrain.corr$isOncogene)] <- 'unclear'


          Yes 
149137  58588 

In [15]:
table(variantTrain.corr$cgType)
sum(is.na(variantTrain.corr))


 Oncogene Recessive   Unknown 
    58588    149137      5933 

[1] 0

In [59]:
save(variantTrain.corr, labelTrain.subset, file='../../../data/trainingData.RData')

In [17]:
head(impact.ann.subset.df$confidence_class)

[1] "AUTO_OK"  "UNLIKELY" "UNLIKELY" "UNLIKELY" "UNLIKELY" "UNLIKELY"

In [18]:
head(labelTrain.subset)

[1]  TRUE FALSE FALSE FALSE FALSE FALSE

In [20]:
table(impact.ann.subset.df$confidence_class, labelTrain.subset)

           labelTrain.subset
             FALSE   TRUE
  AUTO_OK        0 191228
  MANUAL_OK      0   7113
  UNLIKELY   15317      0

In [60]:
table(labelTrain.subset)

labelTrain.subset
 FALSE   TRUE 
 15317 198341 

In [63]:
# by default, include all false mutations
toSubsample <- !labelTrain.subset

In [66]:
# downsample real mutations (vast majority)
realMutationsSelected <- sample(which(labelTrain.subset==TRUE), sum(labelTrain.subset==FALSE))

In [68]:
toSubsample[realMutationsSelected] <- TRUE

In [69]:
sum(toSubsample)

[1] 30634

In [70]:
table(labelTrain.subset[toSubsample])


FALSE  TRUE 
15317 15317 

In [71]:
variantTrain.corr <- variantTrain.corr[toSubsample,]
labelTrain.subset <- labelTrain.subset[toSubsample]

In [76]:
head(impact.ann.subset.df[toSubsample,c('oncokb.id', 'confidence_class')])

,oncokb.id,confidence_class
2,ALK.p.V476A,UNLIKELY
3,PDCD1.p.A215V,UNLIKELY
4,MAP3K1.p.S939C,UNLIKELY
5,FLT4.p.R1324L,UNLIKELY
6,FLT4.p.T494A,UNLIKELY
7,NOTCH4.p.G942R,UNLIKELY


In [80]:
head(impact.ann.subset.df$confidence_class)

[1] "AUTO_OK"  "UNLIKELY" "UNLIKELY" "UNLIKELY" "UNLIKELY" "UNLIKELY"

In [81]:
table(impact.ann.subset.df$confidence_class)


  AUTO_OK MANUAL_OK  UNLIKELY 
   191228      7113     15317 

In [77]:
head(toSubsample)

[1] FALSE  TRUE  TRUE  TRUE  TRUE  TRUE

In [72]:
save(variantTrain.corr, labelTrain.subset, file='../../../data/trainingData.Balanced.RData')